# Predicting churn with the SPSS random tree algorithm

This shows you how to create a predictive model of churn rate by using IBM SPSS Algorithm on Apache Spark version 2.0. You'll learn how to create an SPSS random tree model by using the IBM SPSS Machine Learning API, and how to view the model with IBM SPSS Model Viewer.

Because it consists of multiple classification and regression trees (CART), you can use random tree algorithms to generate accurate predictive models and solve complex classification and regression problems. Each tree develops from a bootstrap sample that is produced by resampling the original data points with replacement data. During the resampling phase, the best split variable is selected for each node from a specified smaller number of variables that are drawn randomly from the full set of variables. Each tree grows without pruning and then, during the scoring phase, the random tree algorithm aggregates tree scores by majority voting (for classification) or average (for regression).

In this notebook, you'll create a model with telecommunications data to predict when its customers will leave for a competitor, so that you can take some action to retain the customer.
    
To get the most out of this notebook, you should have some familiarity with the Scala programming language.

This notebooks runs on Scala 2.11 with Spark 2.0. Some familiarity with Scala is recommended.

## Contents 
This notebook contains the following main sections:

1. [Load the Telco Churn data to the cloud data repository.](#overview)
1. [Prepare the data.](#prepare)
1. [Configure the RandomTrees model.](#configure) 
1. [View the model.](#view)
1. [Summary and next steps.](#next)    

<a id="overview"></a>
## 1. Load the Telco Churn data to the cloud data repository
Telco Churn is a hypothetical data file that concerns a telecommunications company's efforts to reduce turnover in its customer base. Each case corresponds to a separate customer and it records various demographic and service usage information. Before you can work with the data, you must use the URL to get the telco.csv and telco_Feb.csv files from the GitHub repository.


In [15]:
val link_telco = "https://raw.githubusercontent.com/alokkumar70/TelcoChurn/master/Telco.csv"

import sys.process._
import java.net.URL
import java.io.File
new URL(link_telco) #> new File("telco.csv") !!

val link_telco = "https://raw.githubusercontent.com/alokkumar70/TelcoChurn/master/Telco_Feb.csv"

import sys.process._
import java.net.URL
import java.io.File
new URL(link_telco) #> new File("telco_Feb.csv") !!

<a id="prepare"></a>
## 2. Prepare the data

After uploading the CSV files that contain the data, you must create a SQLContext, put the data from the telco.scv file into a Spark DataFrame, and show the first row in the DataFrame.

In [16]:
val sqlContext = new org.apache.spark.sql.SQLContext(sc)

val dfTelco = sqlContext.
    read.
    format("com.databricks.spark.csv").
    option("header", "true").
    option("inferschema", "true").
    load("telco.csv")

dfTelco.show(3)

+---------------+--------------+-------------+-----------------+------------+-------------+--------------+------------+--------------+-------------+---------+
|VOICE_OUT_CALLS|VOICE_OUT_MINS|SMS_OUT_CALLS|OUT_CALLS_ROAMING|GPRS_TRAFFIC|MMS_OUT_CALLS|INTERNET_CALLS|TENURE_START|TENURE_RENEWAL|OUT_COMMUNITY|     ARPU|
+---------------+--------------+-------------+-----------------+------------+-------------+--------------+------------+--------------+-------------+---------+
|     113.719453|   9731.231668|    53.317815|              0.0|         0.0|     2.651429|           0.0|    8.213843|      8.223275|    18.911935|42.977816|
|      85.737091|   5733.881441|    20.173326|         3.729763|         0.0|          0.0|           0.0|   10.939408|     11.208829|    28.696075|30.486712|
|      80.909786|   5299.844355|     3.753237|              0.0|         0.0|          0.0|           0.0|    7.394325|       7.32487|    18.680244|21.567519|
+---------------+--------------+-------------+

Review the data. Print the schema of the DataFrame to look at what kind of data you have.

In [17]:
dfTelco.printSchema

root
 |-- VOICE_OUT_CALLS: double (nullable = true)
 |-- VOICE_OUT_MINS: double (nullable = true)
 |-- SMS_OUT_CALLS: double (nullable = true)
 |-- OUT_CALLS_ROAMING: double (nullable = true)
 |-- GPRS_TRAFFIC: double (nullable = true)
 |-- MMS_OUT_CALLS: double (nullable = true)
 |-- INTERNET_CALLS: double (nullable = true)
 |-- TENURE_START: double (nullable = true)
 |-- TENURE_RENEWAL: double (nullable = true)
 |-- OUT_COMMUNITY: double (nullable = true)
 |-- ARPU: double (nullable = true)



Create a DataFrame for the telco_Feb.csv data. You'll use this year's data to build the model, and use the February data for accuracy value.

In [24]:
val dfTelcoFeb = sqlContext.
    read.
    format("com.databricks.spark.csv").
    option("header", "true").
    option("inferschema", "true").
    load("telco_Feb.csv")
dfTelcoFeb.show(3)

+------+------+---+-------+-------+------+---+------+------+------+------+--------+-----+--------+--------+-------+-------+--------+-------+-------+-------+-------+--------+-------+-------+--------+-----+-----+--------+------+--------+-------+------+-----+----------------+----------------+-------+----------------+----------------+----------------+-------+-----+
|region|tenure|age|marital|address|income| ed|employ|retire|gender|reside|tollfree|equip|callcard|wireless|longmon|tollmon|equipmon|cardmon|wiremon|longten|tollten|equipten|cardten|wireten|multline|voice|pager|internet|callid|callwait|forward|confer|ebill|         loglong|         logtoll|logequi|         logcard|         logwire|           lninc|custcat|churn|
+------+------+---+-------+-------+------+---+------+------+------+------+--------+-----+--------+--------+-------+-------+--------+-------+-------+-------+-------+--------+-------+-------+--------+-----+-----+--------+------+--------+-------+------+-----+----------------

<a id="configure"></a>
## 3. Configure the RandomTrees model

By running this portion of the code, you create the random trees estimator, import the libraries, and set the ordinal and nominal variables. Because no inputFieldList value is set, all fields except the target, frequency, and analysis weight fields are treated as input fields. To make the random tree model build faster, set the number of trees to 10 instead of the default value, which is 100. Finally, you must specify the churn target field. 

In [32]:
import com.ibm.spss.ml.classificationandregression.ensemble.RandomTrees
import com.ibm.spss.ml.utils.DataFrameImplicits.DataFrameEnrichImplicitsClass
val ordinal = Array("ed")
val nominal = Array("region","marital","retire","gender", "tollfree","equip","callcard","wireless","multline","voice","pager","internet","callid","callwait","forward","confer","ebill","custcat","CHURN")

val srf = RandomTrees().setTargetField("churn").setNumTrees(10)
val srfModel = srf.fit(dfTelcoFeb.setNominalMeasure(nominal,true).setOrdinalMeasure(ordinal,true))

Do the prediction and get your results.

In [33]:
val predResult = srfModel.transform(dfTelcoFeb)
val predResultNew = predResult.withColumn("prediction", predResult("prediction").cast("double")).
    withColumn("churn", predResult("churn").cast("double"))

To get the accuracy result, use the Apache Spark **MulticlassClassificationEvaluator** function. Notice that the accuracy is above 90%.

In [8]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
val evaluator = new MulticlassClassificationEvaluator().setLabelCol("churn").setMetricName("accuracy")
val acc_result = evaluator.evaluate(predResultNew)
println(s"acc_result:$acc_result")

acc_result:0.942


<a id="view"></a>
## 4. View the model

View the model with the SPSS Model Viewer. The visualization for the generalized linear model includes a confusion matrix, a table with top decision rules, and a table and chart of predictor importance.

### 4.1 Generate a project token

Before you can run the model viewer, you need to generate a project token

1. In the **My Projects** banner, click the **More** icon and then click **Insert project token**. The project token is inserted into the first cell of the notebook, before the title.
2. Copy the text, which appears at the beginning of the notebook, into the following cell and run it.

In [10]:
// @hidden_cell
// The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
import com.ibm.analytics.projectNotebookIntegration._
val pc = ProjectUtil.newProjectContext(sc, "0863b9f0-8ed3-4fc6-bf1d-5d5bd8f74671", "p-dbbeaf8e0c0f328ee67a791d85409b533b081153")


### 4.2 Start the model viewer

Run the code in the following cell to start SPSS Model Viewer, where you can see a visualization and see model statistics and other characteristics.

In [11]:
import com.ibm.spss.scala.ModelViewer
kernel.magics.html(ModelViewer.toHTML(pc,srfModel))

<!DOCTYPE HTML>
 
 
 
 Model Visualization 
 

 <!-- Global Model Visualization includes -->
 
 
 
 
 
 
 
 
 
 


 
 
 
 
 Open Navigation 
 
 
 
 
 Close Navigation 
 
 
 Random Trees
 
 
 The Random Trees algorithm is a sophisticated modern approach to supervised learning for categorical or continuous targets. The algorithm uses groups of classification or regression trees and randomness to make predictions that are particularly robust when applied to new observations. 
 
 
 
 
 
 EVALUATION 
 Model Evaluation 
 Confusion Matrix 
 MODEL VIEWER 
 Model Information 
 Records Summary 
 Predictor Importance 
 Top Decision Rules 
 
 
 
 

 
   
   
 
 

 
 For classification models, the Model Evaluation panel shows a bar graph displaying the overall prediction accuracy, or proportion of correct predictions, and a table containing a set of evaluation statistics. For regression models, the panel shows a bar graph displaying the R 2 as a measure of prediction accuracy, and a table with a set of evaluation statistics. (In either case if the prediction accuracy is exactly 0, the graph will not be shown.) Accuracy estimates are based on the final ensemble model predictions, not on out-of-bag instances during model building, as is the case with estimates shown in the Model Information Table. ">
 
 
 Model Accuracy 
 
 
 
 
 
 
 
 0.6952573158425832 0.695 
 

 
 

 
 For classification models, the Model Evaluation panel shows a bar graph displaying the overall prediction accuracy, or proportion of correct predictions, and a table containing a set of evaluation statistics. For regression models, the panel shows a bar graph displaying the R 2 as a measure of prediction accuracy, and a table with a set of evaluation statistics. (In either case if the prediction accuracy is exactly 0, the graph will not be shown.) Accuracy estimates are based on the final ensemble model predictions, not on out-of-bag instances during model building, as is the case with estimates shown in the Model Information Table. ">
 Model Evaluation Measures Classification Accuracy 0.695 Weighted True Positive Rate 0.695 Weighted False Positive Rate 0.465 Weighted Precision 0.694 Weighted Recall 0.695 Weighted F 1 Measure 0.695 
 
 

 
 The confusion matrix or classification table contains a cross-classification of observed by predicted labels or groups. The numbers of correct predictions are shown in the cells along the main diagonal. Correct percentages are shown for each row, column and overall. ">
 Confusion Matrix Observed Predicted 1 0 Percent Correct 1 119 153 43.8% 0 149 570 79.3% Percent Correct 44.4% 78.8% 69.5% 
 
 

 
 This table contains information on the type of model fitted, identifies the target field, the number of input features and estimates of accuracy for out-of-bag instances. ">
 Model Information Target Field churn Model Building Method Random Trees Classification Number of Predictors Input 36 Estimated Model Accuracy 0.695 Estimated Misclassification Rate 0.305 
 
 

 
 Shows the number and percentage of records included and excluded from the analysis. ">
 Records Summary Records Number Percent Included 1,000 100.00 Excluded 0 0.00 Total 1,000 100.00 
 
 

 
 This chart displays bars representing the predictors in descending order of relative importance for predicting the target. The values for each predictor are scaled so that they add to 1. ">
 
 
 
 
 
 
 

 
 
 
 
 
 
 tenure 
 Importance 
 .098 
 
 
 
 
 
 Mean 35.526 
 Standard Deviation 21.349 
 Minimum 1.000 
 Maximum 72.000 
 N 1,000 
 
 
 
 
 
 
 
 age 
 Importance 
 .093 
 
 
 
 
 
 Mean 41.684 
 Standard Deviation 12.553 
 Minimum 18.000 
 Maximum 77.000 
 N 1,000 
 
 
 
 
 
 
 
 address 
 Importance 
 .079 
 
 
 
 
 
 Mean 11.551 
 Standard Deviation 10.082 
 Minimum .000 
 Maximum 55.000 
 N 1,000 
 
 
 
 
 
 
 
 employ 
 Importance 
 .069 
 
 
 
 
 
 Mean 10.987 
 Standard Deviation 10.077 
 Minimum .000 
 Maximum 47.000 
 N 1,000 
 
 
 
 
 
 
 
 region 
 Importance 
 .061 
 
 
 


### 4.3. Export the XML files (PMML, StatXML) for other detail statistics
By exporting your results to different formats, such as Predictive Model Markup Language (PMML) or statXML format you can share your statistical analyses outside of IBM Data Science Experience.

In [12]:
import java.io.{File, PrintWriter}

srfModel.toPMML("randomTrees_pmml.xml")
val statXML = srfModel.statXML()
new PrintWriter("StatXML.xml") {
      write(statXML)
      close
}

$anon$1@5f613d99

<a id="next"></a>
# Summary and next steps
You have created a predictive model of churn rate by using IBM SPSS Algorithm on Apache Spark. Now you can create a different model to compare model evaluations, such as the test of model effects, residuals, and so on. See [SPSS documentation](https://apsportal.ibm.com/docs/content/kc_gen/integrations-gen2.html).

## Authors

Wang Zhiyuan and Yu Wenpei are SPSS Algorithm Engineers at IBM.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.